In [ ]:
import gym
from gym import error,spaces,utils
from gym.utils import seeding
from typing import Optional
from collections import deque

import os
import pybullet as p
import pybullet_data
from pybullet_utils import bullet_client as bc
import math
import numpy as np
import random
import time

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import History

# Environment

In [ ]:

DEFAULT_X = np.pi
DEFAULT_Y = 1.0

class pendulumEnv(gym.Env):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 30,
    }
    def __init__(self, render_mode: Optional[str] = None, g=9.81):
        self.max_speed = 8
        self.max_torque = 0.15
        self.dt = 0.25
        self.g = g
        self.m = 0.4
        self.l = 0.08
        self._render_height = 200
        self._render_width = 320
        self._physics_client_id = -1

        self.render_mode = render_mode

        self.screen_dim = 500
        self.screen = None
        self.clock = None
        self.isopen = True

        high = np.array([1.0, 1.0, self.max_speed], dtype=np.float32)
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
    def step(self,u,step):
        p = self._p
        th, thdot = self.state  

        g = self.g
        
        m = self.m
        l = self.l
        dt = self.dt

        if u==0:
            u = -0.15
        elif u==2:
            u = 0.15
        elif u==1:
            u = 0
            
        self.last_u = u  
        
        p.setJointMotorControl2(self.boxId,0,p.TORQUE_CONTROL,force=u)
        p.stepSimulation()
    
        costs = -angle_norm(th) + 0.01*abs(thdot)
        

        
        th,thdot = p.getJointState(self.boxId,0)[0:2]
        self.state = th,thdot

        if self.render_mode == "human":
            self.render()
        return self._get_obs(), -costs, False, False, {}
    
    def _get_obs(self):
        theta, thetadot = self.state
        return np.array([np.cos(theta), np.sin(theta), thetadot], dtype=np.float32)
        
        
    def reset(self):
        if self._physics_client_id < 0:
            if self.render_mode == "human":
                self._p = bc.BulletClient(connection_mode=p.GUI)
            else:
                self._p = bc.BulletClient()
                
            self._physics_client_id = self._p._client
                
            p2 = self._p
            p2.resetSimulation()
            urdfRootPath=pybullet_data.getDataPath()
            planeUid = p2.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
            cubeStartPos = [0,0,1]
            cubeStartOrientation = p2.getQuaternionFromEuler([0,0,0])
            self.boxId = p2.loadURDF(os.path.join(urdfRootPath,"sreeram/Pendulum.urdf"),cubeStartPos, cubeStartOrientation)
        
            self.timeStep = 0.25
            p2.setJointMotorControl2(self.boxId, 0, p2.VELOCITY_CONTROL, force=0.01)
            p2.setGravity(0, 0, -9.8)
            p2.setTimeStep(self.timeStep)
            p2.setRealTimeSimulation(0)
            
            
        p2 = self._p
        p2.resetJointState(self.boxId, 0, 0, 0)
        self.state = p2.getJointState(self.boxId, 0)[0:2] 
        return self._get_obs()
    
    def _get_obs(self):
        theta, thetadot = self.state
        return np.array([np.cos(theta), np.sin(theta), thetadot], dtype=np.float32)
        
    
    def render(self,mode="human",close=False):
        if mode=="human":
            self._renders=True
        if mode!="rgb_array":
            return np.array([])
    
        base_pos=[0,0,0]
        self._cam_dist = 2
        self._cam_pitch = 0.3
        self._cam_yaw = 0
        if (self._physics_client_id>=0):
            view_matrix = self._p.computeViewMatrixFromYawPitchRoll(
            cameraTargetPosition=base_pos,
            distance=self._cam_dist,
            yaw=self._cam_yaw,
            pitch=self._cam_pitch,
            roll=0,
            upAxisIndex=2)
            proj_matrix = self._p.computeProjectionMatrixFOV(fov=60,
                 aspect=float(self._render_width) /
                 self._render_height,
                 nearVal=0.1,
                 farVal=100.0)
            (_, _, px, _, _) = self._p.getCameraImage(
              width=self._render_width,
              height=self._render_height,
              renderer=self._p.ER_BULLET_HARDWARE_OPENGL,
              viewMatrix=view_matrix,
              projectionMatrix=proj_matrix)
        else:
            px = np.array([[[255,255,255,255]]*self._render_width]*self._render_height, dtype=np.uint8)
        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(np.array(px), (self._render_height, self._render_width, -1))
        rgb_array = rgb_array[:, :, :3]
        return rgb_array

def angle_normalize(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi

def angle_norm(x):
    return abs((((x + np.pi) % (2 * np.pi)) - np.pi)**2)-(np.pi**2)

In [ ]:
env = pendulumEnv(render_mode="human")

print(env.action_space.n)

num_actions = env.action_space.n
num_states = env.observation_space.shape[0]
print("Size of State Space ->  {}".format(num_states))


# DDQN Agent

In [ ]:
MAX_EPSILON = 1
MIN_EPSILON = 0.01

GAMMA = 0.95
LAMBDA = 0.0005
TAU = 0.08

BATCH_SIZE = 32
REWARD_STD = 1.0

action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
sine_angle_history = []
cos_angle_history = []
velocity_history = []

In [ ]:
class ExpirienceReplay:
    def __init__(self, maxlen = 2000):
        self._buffer = deque(maxlen=maxlen)
    
    def store(self, state, action, reward, next_state, terminated):
        self._buffer.append((state, action, reward, next_state, terminated))
        
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(next_state)
        done_history.append(terminated)
        rewards_history.append(reward)
        sine_angle_history.append(state[1])
        cos_angle_history.append(state[0])
        velocity_history.append(state[2])
              
    def get_batch(self, batch_size):
        if batch_size > len(self._buffer):
            return random.sample(self._buffer, len(self._buffer))
        else:
            return random.sample(self._buffer, batch_size)
        
    def get_arrays_from_batch(self, batch):
        states = np.array([x[0] for x in batch])
        actions = np.array([x[1] for x in batch])
        rewards = np.array([x[2] for x in batch])
        next_states = np.array([(np.zeros(NUM_STATES) if x[3] is None else x[3]) 
                                for x in batch])
        
        return states, actions, rewards, next_states
        
    def buffer_size(self):
        return len(self._buffer)

In [ ]:
class DDQNAgent:
    def __init__(self, expirience_replay, state_size, actions_size, optimizer):
        
        # Initialize atributes
        self._state_size = state_size
        self._action_size = actions_size
        self._optimizer = optimizer
        
        self.expirience_replay = expirience_replay
        
        # Initialize discount and exploration rate
        self.epsilon = MAX_EPSILON
        
        # Build networks
        self.primary_network = self._build_network()
        self.primary_network.compile(loss='mse', optimizer=self._optimizer)
        self.target_network = self._build_network()   
   
    def _build_network(self):
        last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
        network = Sequential()
        network.add(Dense(30, activation='relu',input_shape=(self._state_size,)))
        network.add(Dense(30, activation='relu'))
        network.add(Dense(self._action_size,kernel_initializer=last_init,activation="linear"))
        
        return network
    
    def align_epsilon(self, step):
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * step)
    
    def align_target_network(self):
        for t, e in zip(self.target_network.trainable_variables, 
                    self.primary_network.trainable_variables): t.assign(t * (1 - TAU) + e * TAU)
    
    def act(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, self._action_size)
        else:
            q_values = self.primary_network(state.reshape(1, -1))
            return np.argmax(q_values)
    
    def store(self, state, action, reward, next_state, terminated):
        self.expirience_replay.store(state, action, reward, next_state, terminated)
    
    def train(self, batch_size):
        if self.expirience_replay.buffer_size() < BATCH_SIZE * 3:
            return 0
        
        batch = self.expirience_replay.get_batch(batch_size)
        states, actions, rewards, next_states = expirience_replay.get_arrays_from_batch(batch)
        
        # Predict Q(s,a) and Q(s',a') given the batch of states
        q_values_state = self.primary_network(states).numpy()
        q_values_next_state = self.primary_network(next_states).numpy()
        
        # Copy the q_values_state into the target
        target = q_values_state
        updates = np.zeros(rewards.shape)
                
        valid_indexes = np.array(next_states).sum(axis=1) != 0
        batch_indexes = np.arange(BATCH_SIZE)

        action = np.argmax(q_values_next_state, axis=1)
        q_next_state_target = self.target_network(next_states)
        updates[valid_indexes] = rewards[valid_indexes] + GAMMA * q_next_state_target.numpy()[batch_indexes[valid_indexes], action[valid_indexes]]
        
        target[batch_indexes, actions] = updates
        loss = self.primary_network.train_on_batch(states, target)

        # update target network parameters slowly from primary network
        self.align_target_network()
        
        return loss


In [ ]:
class AgentTrainer():
    def __init__(self, agent, enviroment):
        self.agent = agent
        self.enviroment = enviroment
        
    def _take_action(self, action,max_steps):
        next_state, reward, terminated, _, temp = self.enviroment.step(action,max_steps) 
        next_state = next_state if not terminated else None
        #reward = np.random.normal(1.0, REWARD_STD)
        return next_state, reward, terminated
    
    def _print_epoch_values(self, episode, total_epoch_reward, average_loss,ep_reward):
        print("**********************************")
        print(f"Episode: {episode} - Reward: {ep_reward} - Average Loss: {average_loss:.3f}")
    
    def train(self, num_of_episodes = 500):
        total_timesteps = 0  
        
        for episode in range(0, num_of_episodes):

            # Reset the enviroment
            state = self.enviroment.reset()

            # Initialize variables
            average_loss_per_episode = []
            average_loss = 0
            total_epoch_reward = 1
            max_steps=0
            ep_reward=0

            terminated = False

            while max_steps<500:

                action = agent.act(state)

                # Take action    
                next_state, reward, terminated = self._take_action(action,max_steps)
                agent.store(state, action, reward, next_state, terminated)
                
                
                
                loss = agent.train(BATCH_SIZE)
                average_loss += loss

                state = next_state
                agent.align_epsilon(total_timesteps)
                total_timesteps += 1
                max_steps+=1
                ep_reward+=reward
                
                
            
            average_loss /= total_epoch_reward
            average_loss_per_episode.append(average_loss)
            self._print_epoch_values(episode, total_epoch_reward, average_loss,ep_reward)     
            total_epoch_reward +=1

In [ ]:
optimizer = Adam()
expirience_replay = ExpirienceReplay(100000)
agent = DDQNAgent(expirience_replay, num_states, num_actions, optimizer)

In [ ]:
# Training
agent_trainer = AgentTrainer(agent, env)
agent_trainer.train()


In [ ]:
"""
# Testing

for episode in range(0, 5):
    state = env.reset()

    average_loss_per_episode = []
    average_loss = 0
    total_epoch_reward = 1
    max_steps=0
    ep_reward=0

    terminated = False

    while max_steps<500:

        action = np.argmax(agent.primary_network(state.reshape(1,-1)))
        
        next_state, reward, terminated, _ , temp = env.step(action,max_steps)

        state = next_state
        max_steps+=1
        ep_reward+=reward
                
    print("Episode {} reward is {}".format(episode,ep_reward))
"""